# LlamaIndex Bottoms-Up Development - LLMs and Prompts
This notebook walks through testing an LLM using the primary prompt templates used in llama-index.

In [2]:
import openai
import os

# openai.api_key = "YOUR_API_KEY"
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

os.environ["OPENAI_API_KEY"] = "Not Needed"
os.environ["OPENAI_API_BASE"] = "http://localhost:1234/v1"

## Setup
In this section, we load a test document, create an LLM, and copy prompts from llama-index to test with.

First, let's load a quick document to test with. Right now, we will just load it as plain text, but we can do other operations later!

In [3]:
with open("../docs/getting_started/starter_example.md", "r") as f:
    text = f.read()

Next, we create our LLM!

In [4]:
from llama_index.llms import OpenAI
# llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

# llm = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")
llm = OpenAI()


LlamaIndex uses some simple templates under the hood for answering queries -- mainly a `text_qa_template` for obtaining initial answers, and a `refine_template` for refining an existing answer when all the text does not fit into one LLM call.

Let's copy the default templates, and test out our LLM with a few questions.

In [5]:
from llama_index import Prompt

text_qa_template = Prompt(
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)

refine_template = Prompt(
    "We have the opportunity to refine the original answer "
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "Given the new context, refine the original answer to better "
    "answer the question: {query_str}. "
    "If the context isn't useful, output the original answer again.\n"
    "Original Answer: {existing_answer}"
)

Now, lets test a few questions!

## Text QA Template Testing

In [11]:
question = "How can I install llama-index?"
prompt = text_qa_template.format(context_str=text, query_str=question)
response = llm.complete(prompt)
print(response.text)

To install llama-index, you need to follow these steps:
1. Clone the repository using `git clone https://github.com/jerryjliu/llama_index.git`.
2. Navigate to the cloned directory using `cd llama_index`.
3. Install the dependencies by running `pip install -r requirements.txt` or `poetry install`.
4. You can now use llama-index in your Python scripts or Jupyter notebooks by importing it as `from llama_index import *`.


In [12]:
question = "How do I create an index?"
prompt = text_qa_template.format(context_str=text, query_str=question)
response = llm.complete(prompt)
print(response.text)

To create an index using LlamaIndex, follow these steps:

1. First, make sure you have installed LlamaIndex by following the [installation](installation.md) instructions.
2. Navigate to the `examples` folder of the LlamaIndex repository and open the desired example directory in your terminal or command prompt. For instance, if you want to use the Paul Graham essay example, go to the `examples/paul_graham_essay` directory.
3. Create a new Python file (e.g., `create_index.py`) and import the necessary modules:
```python
from llama_index import VectorStoreIndex, SimpleDirectoryReader
```
4. Load the documents you want to index using `SimpleDirectoryReader`. In this case, it's assumed that the documents are in a `data` folder within the example directory:
```python
documents = SimpleDirectoryReader('data').load_data()
```
5. Create an instance of `VectorStoreIndex` from the loaded documents:
```python
index = VectorStoreIndex.from_documents(documents)
```
6


In [6]:
question = "How do I create an index? Write your answer using only code."
prompt = text_qa_template.format(context_str=text, query_str=question)
response_gen = llm.stream_complete(prompt)
for response in response_gen:
    print(response.delta, end="")


To create an index using LlamaIndex, follow these steps:

1. Import the necessary modules:
```python
from llama_index import VectorStoreIndex, SimpleDirectoryReader
```

2. Load the documents into memory:
```python
documents = SimpleDirectoryReader('data').load_data()
```
Replace `'data'` with the path to your data folder.

3. Create a new index from the loaded documents:
```python
index = VectorStoreIndex.from_documents(documents)
```

4. (Optional) Save the index to disk for later use:
```python
index.storage_context.persist()
```

5. To load an index from disk, first create a new `StorageContext` and then load the index using `load_index_from_storage`:
```python
from llama_index import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./storage")
# load index
index = load_index_from_

## Refine Template Testing

In [7]:
question = "How do I create an index? Write your answer using only code."
existing_answer = """To create an index using LlamaIndex, you need to follow these steps:

1. Download the LlamaIndex repository by cloning it from GitHub.
2. Navigate to the `examples/paul_graham_essay` folder in the cloned repository.
3. Create a new Python file and import the necessary modules: `VectorStoreIndex` and `SimpleDirectoryReader`.
4. Load the documents from the `data` folder using `SimpleDirectoryReader('data').load_data()`.
5. Build the index using `VectorStoreIndex.from_documents(documents)`.
6. To persist the index to disk, use `index.storage_context.persist()`.
7. To reload the index from disk, use the `StorageContext` and `load_index_from_storage` functions.

Note: This answer assumes that you have already installed LlamaIndex and have the necessary dependencies."""
prompt = refine_template.format(context_msg=text, query_str=question, existing_answer=existing_answer)
response = llm.complete(prompt)
print(response.text)

To create an index using LlamaIndex, follow these steps in a Python file:

```python
# Import required modules
from llama_index import VectorStoreIndex, SimpleDirectoryReader

# Load documents from data folder
documents = SimpleDirectoryReader('data').load_data()

# Build the index
index = VectorStoreIndex.from_documents(documents)

# Persist the index to disk (optional)
index.storage_context.persist()
```

This code snippet demonstrates how to create an index using LlamaIndex by loading documents from a `data` folder and building the index with `VectorStoreIndex.from_documents(documents)`. If you want to persist the index to disk, use `index.storage_context.persist()`. To reload the index from disk, use the following code:

```python
# Rebuild storage context and load index
from llama_index import StorageContext, load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir="./storage")
index = load_index_from_storage(storage


### Chat Example
The LLM also has a `chat` method that takes in a list of messages, to simulate a chat session. 

In [8]:
from llama_index.llms import ChatMessage

chat_history = [
    ChatMessage(role="system", content="You are a helpful QA chatbot that can answer questions about llama-index."),
    ChatMessage(role="user", content="How do I create an index?"),
]

response = llm.chat(chat_history)
print(response.message)

assistant:  To create an index using Llama Index, you'll need to follow these steps:

1. Install the required packages: First, make sure you have Python and the necessary libraries installed on your system. You can install Llama Index by running `pip install llama-index` in your terminal or command prompt.

2. Import the required modules: In your Python script or Jupyter Notebook, import the `GPTSimpleVectorIndex` class from the `llama_index` package. You can do this with the following line of code: `from llama_index import GPTSimpleVectorIndex`.

3. Prepare your data: Llama Index works with text documents or web pages, so you'll need to convert your data into a format that it can understand. This usually involves tokenizing and encoding the text using a language model like OpenAI's GPT-2 or GPT-Neo. You can use the `LLMIndex` class from the `llama_index` package to create an index from a list of documents, where each document is a string or a dictionary with a 'text' key.

4. Initiali

## Conclusion

In this notebook, we covered the low-level LLM API, and tested out some basic prompts with out documentation data.